# Práctica Python - Tecnolgías de Procesamiento Big Data

+ Autor: Felipe Navarro
+ Fecha: 29/11/2020

## Ejercicio 3

In [1]:
import pandas as pd
import numpy as np
import scipy

from scipy.optimize import minimize, LinearConstraint


In [2]:
df_curvas_roi = pd.read_csv("datos_curvas_roi.csv",header=0)

In [3]:
df_curvas_roi.head()

,MEDIO,A,B,MIN,MAX,decay
0,Televisión,7.091115,270163.85990,183359.57680,421967.23580,0.40
1,Online,6.571585,62164.76381,38488.03122,110009.09710,0.20
2,Prensa,6.779517,105752.53950,71820.20725,165047.65840,0.20
3,Radio,6.182705,64579.77595,43419.72799,102131.35590,0.15
4,Exterior,4.168857,26693.37017,20854.44233,34971.98656,0.30


In [4]:

x_estudio =np.array([
    [183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680],
    [38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122],
    [71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725],
    [43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799],
    [20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233]
    
])

In [5]:
# Vector de inversión ejemplo, para validar la función
x_ej = np.array([
    [183359.57680,183359.57680,183359.57680,183359.57680],
    [38488.03122,38488.03122,38488.03122,38488.03122],
    [71820.20725,71820.20725,71820.20725,71820.20725],
    [43419.72799,43419.72799,43419.72799,43419.72799],
    [20854.44233,20854.44233,20854.44233,20854.44233]]
)

In [6]:
def efecto_adstock(x,decay):
    """
    Función que recibe un vector de inversión en distintos medios y un vector de decays para cada medio y calcula
    la inversión ajustada por el efecto adstock
    
    Params:
        x (array): vector de inversiones en cada medio
        decay (array): vector de factor decay para cada medio
        
    Returns:
        x_hat (array): vector de inversión para cada medio, ajustado con el efecto adstock
    
    """
    x_hat = []
    
    for i in range(len(x)):
        y_hat = []
        for j in range(len(x[i])):
            if j==0:
                y_hat.append(x[i][j])
            else:    
                y_hat.append(y_hat[j-1]*decay[i]+x[i][j])
        x_hat.append(y_hat)
        
    return x_hat

In [7]:
vect_decay = np.array(df_curvas_roi["decay"])

In [8]:
x_hat = efecto_adstock(x_ej,vect_decay)

In [9]:
x_hat

[[183359.5768, 256703.40752, 286040.939808, 297775.9527232],
 [38488.03122, 46185.637464, 47725.158712799996, 48033.06296256],
 [71820.20725, 86184.24870000001, 89057.05699000001, 89631.618648],
 [43419.72799, 49932.6871885, 50909.631068275, 51056.17265024125],
 [20854.44233, 27110.775029000004, 28987.674838700004, 29550.744781610003]]

Una vez validada la función que calcula el efecto adstock, se pasa a definir la función que calcula el aporte total

In [10]:
def aportes_totales(x):
    """
    Función para calcular los aportes totales utilizando el efecto adstock y las curvas de aporte
    
    Params:
        x (array): vector de inversiones en cada medio
        
    Returns:
        aporte_total (float): Aporte total de toda la inversión
    """
    x_hat = efecto_adstock(x,vect_decay)
    aporte_total = []
    for i in range(len(x_hat)):
        a = df_curvas_roi["A"][i]
        b = df_curvas_roi["B"][i]
        f_x = 0
        for j in range(len(x_hat[i])):
            f_x += np.exp(a-(b/x_hat[i][j]))
        aporte_total.append(f_x)
        
    aporte_total = np.sum(aporte_total)
    return aporte_total

In [12]:
aporte_test = aportes_totales(x_ej)

In [13]:
aporte_test

3972.248270860605

Una vez validada esta segunda función, se calcula con los datos para el estudio de 10 semanas

In [14]:
aportes_estudio = aportes_totales(x_estudio)

In [15]:
aportes_estudio

10723.620391631033

## Parte de optimización

Para esta segunda parte, he tenido que modificar la función aportes_totales para pasarle un vector y después dividirlo en sub-vectores para cada medio

No consigo que la solución converja. Probablemente haya definido mal los constraints. No obstante, dejo el código que he probado

In [16]:
def chunks(lst, n):
    matrix = [list(t) for t in zip(*[iter(lst)]*n)]
    return matrix

In [17]:
def aportes_totales(x):
    """
    Función para calcular los aportes totales utilizando el efecto adstock y las curvas de aporte
    
    Params:
        x (array): vector de inversiones en cada medio
        
    Returns:
        aporte_total (float): Aporte total de toda la inversión
    """
    x = chunks(x, 10)
    x_hat = efecto_adstock(x,vect_decay)
    aporte_total = []
    
    for i in range(len(x_hat)):
        a = df_curvas_roi["A"][i]
        b = df_curvas_roi["B"][i]
        f_x = 0
        for j in range(len(x_hat[i])):
            f_x += np.exp(a-(b/x_hat[i][j]))
        aporte_total.append(f_x)
        
    aporte_total = np.sum(aporte_total)
    return aporte_total

In [31]:
# Vector de inversión 
x0 =np.array(
    [183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,183359.57680,
    38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,38488.03122,
    71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,71820.20725,
    43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,43419.72799,
    20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233,20854.44233])

In [20]:
a = [a for a in df_curvas_roi["A"]]

In [21]:
b = [b for b in df_curvas_roi["B"]]

In [22]:
limites = zip(a,b)

In [ ]:
bounds = [i for i in limites]

In [32]:
bounds = [
    (7.091114787, 270163.8599),(7.091114787, 270163.8599),(7.091114787, 270163.8599),(7.091114787, 270163.8599),(7.091114787, 270163.8599),(7.091114787, 270163.8599),(7.091114787, 270163.8599),(7.091114787, 270163.8599),(7.091114787, 270163.8599),(7.091114787, 270163.8599),
     (6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),(6.571585223, 62164.763810000004),
     (6.779516794, 105752.5395),(6.779516794, 105752.5395),(6.779516794, 105752.5395),(6.779516794, 105752.5395),(6.779516794, 105752.5395),(6.779516794, 105752.5395),(6.779516794, 105752.5395),(6.779516794, 105752.5395),(6.779516794, 105752.5395),(6.779516794, 105752.5395),
     (6.18270522, 64579.77595),(6.18270522, 64579.77595),(6.18270522, 64579.77595),(6.18270522, 64579.77595),(6.18270522, 64579.77595),(6.18270522, 64579.77595),(6.18270522, 64579.77595),(6.18270522, 64579.77595),(6.18270522, 64579.77595),(6.18270522, 64579.77595),
     (4.168857373, 26693.37017),(4.168857373, 26693.37017),(4.168857373, 26693.37017),(4.168857373, 26693.37017),(4.168857373, 26693.37017),(4.168857373, 26693.37017),(4.168857373, 26693.37017),(4.168857373, 26693.37017),(4.168857373, 26693.37017),(4.168857373, 26693.37017)
]



In [33]:
budget = 6000000

In [36]:
cons = ({'type':'ineq', 'fun':lambda x: np.sum(x)-budget}) 

In [37]:
sol = minimize(

    aportes_totales,
    x0,
    method='SLSQP',
    constraints=cons,
    bounds = bounds,
    options = {'maxiter' : 1000000}

)
sol



     fun: 15610.429801916158
     jac: array([0.00231934, 0.00170898, 0.00158691, 0.00146484, 0.00146484,
       0.00146484, 0.00146484, 0.00134277, 0.0012207 , 0.00085449,
       0.00512695, 0.00439453, 0.00427246, 0.00427246, 0.00427246,
       0.00415039, 0.00415039, 0.00415039, 0.00402832, 0.0032959 ,
       0.00366211, 0.00305176, 0.00292969, 0.00305176, 0.00292969,
       0.00292969, 0.00305176, 0.00305176, 0.00292969, 0.00244141,
       0.00317383, 0.00280762, 0.00268555, 0.00268555, 0.00268555,
       0.00268555, 0.00268555, 0.00268555, 0.00268555, 0.00231934,
       0.00109863, 0.00085449, 0.00085449, 0.00085449, 0.00085449,
       0.00085449, 0.00073242, 0.00073242, 0.00073242, 0.00061035])
 message: 'Positive directional derivative for linesearch'
    nfev: 30924
     nit: 620
    njev: 604
  status: 8
 success: False
       x: array([270163.8599 , 270163.8599 , 270163.8599 , 270163.8599 ,
       270163.8599 , 270163.8599 , 270163.8599 , 270163.8599 ,
       270163.8599 , 27